# Co-occurrence network analysis tutorial

In this notebook we will illustrate how interactive exploration and analysis of the [CORD-19](https://www.kaggle.com/allen-institute-for-ai/CORD-19-research-challenge) dataset can be performed using the `cord19kg` package. The exploration and analysis techniques presented here focus on named entities and their co-occurrence in the scientific articles constituting the dataset.

The input data for this notebook contains the named entities extracted from a small selection of 20 articles representing 10 different entity types (i.e. proteins, chemicals, drugs, diseases, condtions, organs, organisms, pathways, cell types, cell compartments). The article selection corresponds to 20 most relevant articles to the query _"Glucose is a risk factor of COVID-19"_ obtained using the article search model [ref to blue brain search](). The entity extraction was performed using the Named Entity Recognition (NER) techniques also included in [ref to blue brain search]().

The `cord19kg` package provides a set of tools for interactive literature exploration through the named entity co-occurrence analysis consisting of the following steps:

1. __Data preparation__ step converts raw mentions into aggregated entity occurrence statistics.
2. __Data curation__ step allows the user to manage extracted entities: modify, filter them and link to the ontology.
3. __Network generation__ step allows creating entity co-occurrence networks based on paper-, section- and paragraph-level co-occurrence relations between entities. These entity relations are quantified using mutual-information-based scores (pointwise mutual information and its normalized version).
4. __Network visualization and analysis__ step allows the user to perform interactive network visualization, edit network elements and perform its analysis (spanning tree, mutual-information based shortest paths between entities, etc).

In [1]:
import json
import zipfile

import pandas as pd

import dash_cytoscape as cyto

from cord19kg.utils import (generate_curation_table,
                            link_ontology,
                            generate_cooccurrence_analysis)
from cord19kg.apps.curation_app import curation_app
# from cord19kg.apps.visualization_app import visualization_app

The cell below loads additional graph layouts used in the graph visualization app

In [2]:
cyto.load_extra_layouts()

## 1. Data preparation

The input dataset contains occurrences of different terms in paragraphs of scientific articles from the CORD-19 dataset previously extracted by means of a NER model.

In [3]:
data = pd.read_csv("../data/Glucose_risk_20_papers.csv")

In [4]:
data.sample(5)

,entity,entity_type,occurrence
3150,human,ORGANISM,214924:Angiotensin-Converting Enzyme 2 Express...
1255,ADA,CHEMICAL,184360:Cardiovascular Effects Of Sdpp4 Upregul...
3420,Nile Virus,ORGANISM,214924:The Interplay Between Covid-19 And Ampk...
214,COVID-19,DISEASE,151932:Abstract:1
547,pulmonary,ORGAN,179426:Role Of Antidiabetic Drugs In Current C...


On the first preparation step, we group and aggregate the input data by unique entities.

In [5]:
%%time
print("Prepating curatation data...")
curation_input_table, factor_counts = generate_curation_table(data)
print("Done.")

Prepating curatation data...
Cleaning up the entities...
Aggregating occurrences of entities....
Done.
CPU times: user 289 ms, sys: 5.66 ms, total: 295 ms
Wall time: 294 ms


The resulting dataframe contains a row per unique named entity together with the following occurrence data: 
- sets of unique paragraphs, papers, sections, where the corresponding entity is mentioned (`paper`, `section`, `paragraph` columns);
- number of total entity occurrences (the `raw_frequency` column);
- number of unique papers where it occurs (the `paper_frequency` column);
- unique entity types assigned by the NER model (the `entity_type` column, multiple types are possible).
- raw entity types assigned by the NER model with the multiplicity of thier occurrence (the `raw_entity_types` column).


In [6]:
curation_input_table.sample(5)

,entity,entity_type,paragraph,paper,section,paper_frequency,raw_entity_types,raw_frequency
121,human,ORGANISM,[184360:Pathophysiology Of Covid-19: Pulmonar ...,"[211373, 211125, 214924, 184360]","[184360:Sdpp4 As Soluble Decoy Factor , 184360...",4,"[ORGANISM, ORGANISM, ORGANISM, ORGANISM, ORGAN...",27
54,chest pain,DISEASE,"[214924:Introduction:6, 179426:Measures For Go...","[179426, 214924]","[214924:Introduction, 179426:Measures For Good...",2,"[DISEASE, DISEASE]",2
206,pneumonia,DISEASE,"[160564:Introduction:3, 214728:Cap Community-A...","[214728, 214924, 114119, 197804, 179426, 18436...","[214924:Introduction, 114119:Abstract, 214728:...",8,"[DISEASE, DISEASE, DISEASE, DISEASE, DISEASE, ...",17
208,prognosis,DISEASE,"[35198:Title:0, 104188:Title:0]","[35198, 104188]","[104188:Title, 35198:Title]",2,"[DISEASE, DISEASE]",2
104,gliptins,PROTEIN,[184360:Gliptins ::: Therapeutic Potential Of ...,[184360],"[184360:Gliptins , 184360:Caption, 184360:Conc...",1,"[PROTEIN, PROTEIN, PROTEIN, PROTEIN, PROTEIN, ...",17


The second output of the data preparation step outputs the counts of different instances of occurrence factors: number of distinct papers/sections/paragraphs in the input corpus.

In [7]:
factor_counts

{'paper': 20, 'section': 108, 'paragraph': 286}

## 2. Data curation

### Loading the NCIT ontology linking data

To group synonymical entities in the previously extracted table (e.g. `ace2`, `ace-2`, `angiotensin-converting enzyme 2`), as well as assign additional semantics to these entities (e.g. human-readable definition, taxonomy, etc), we peform further _linking_ of the entities to the terms from the [NCIT ontology](https://ncithesaurus.nci.nih.gov/ncitbrowser/).

To be able to perform such ontology linking, we load some additional (pre-computed using ML-based linking models) data.

In [8]:
%%time
print("Loading the ontology linking data...")
    
print("\tDecompressing the input data file...")
with zipfile.ZipFile("../data/NCIT_ontology_linking_3000_papers.csv.zip", 'r') as zip_ref:
    zip_ref.extractall("../data/")

print("\tLoading the linking dataframe in memory...")
ontology_linking = pd.read_csv("../data/NCIT_ontology_linking_3000_papers.csv")

print("\tLoading ontology type mapping...")
with open("../data/NCIT_type_mapping.json", "rb") as f:
    type_mapping = json.load(f)
print("Done.")

Loading the ontology linking data...
	Decompressing the input data file...
	Loading the linking dataframe in memory...
	Loading ontology type mapping...
Done.
CPU times: user 1.02 s, sys: 164 ms, total: 1.19 s
Wall time: 1.2 s


The ontology linking table contains the following columns:
- `mention` entity mentioned in the text
- `concept` ontology concept linked to the entity mention
- `uid` unique identifier of the ontology concept
- `definition` definition of the concept
- `taxonomy` taxonomy of semantic types associated with the concept

In [9]:
ontology_linking.sample(5)

,mention,concept,uid,definition,taxonomy
27549,apical bone,apical,http://purl.obolibrary.org/obo/NCIT_C25423,Relating to or located at the tip (an apex).,[('http://purl.obolibrary.org/obo/NCIT_C13442'...
133975,palpationsystemic,palpation,http://purl.obolibrary.org/obo/NCIT_C16950,A method of feeling with the hands during a ph...,[('http://purl.obolibrary.org/obo/NCIT_C20989'...
47298,brucella-associated glomerulonephritis,glomerulonephritis,http://purl.obolibrary.org/obo/NCIT_C26784,A renal disorder characterized by damage in th...,[('http://purl.obolibrary.org/obo/NCIT_C26833'...
81165,antineoplastic,antineoplastic agent,http://purl.obolibrary.org/obo/NCIT_C274,"A substance that inhibits the maturation, grow...","[('http://purl.obolibrary.org/obo/NCIT_C1909',..."
34590,itch,pruritus,http://purl.obolibrary.org/obo/NCIT_C3344,An intense itching sensation.,"[('http://purl.obolibrary.org/obo/NCIT_C5036',..."


### Interactive curation of  entity occurrence data

The package provides an interactive entity curation app that allows the user to visualize the entity occurrence data, modify it, perform ontology linking (see `Link to NCIT ontology` button), filter short or unfrequent entities.

The field `Keep` allows specifying a set of entities that must be kept in the dataset at all times (even if they don't satisfy the selected filtering criteria).

Finally the value specified in the `Generate Graphs from top 500 frequent entities` field corresponds to the number of top entities (by the frequency of their occurrence in papers) to be included in the co-occurrence network.

We load the prepared data table into the curation app as follows:

In [10]:
curation_app.set_table(curation_input_table.copy())

We can specify the default entities to keep.

In [11]:
default_entities_to_keep = ["glucose", "covid-19"]
curation_app.set_default_terms_to_include(default_entities_to_keep)

Finally, we set the ontology linking callback to be fired upon a click on the `Link to NCIT ontology` button.

In [12]:
curation_app.set_ontology_linking_callback(lambda x: link_ontology(ontology_linking, type_mapping, x))

### Launch the curation app

The application can be launched either inline (inside the current notebook) as below.

In [13]:
curation_app.run(port=8072, mode="inline")

Merging the occurrence data with the ontology linking...


Or it can be opened externally (by the URL that you can open in a separate tab of your browser, try uncommenting, executing and doing Ctrl+Click on the displayed URL).

In [14]:
# curation_app.run(port=8070, mode="external")

## 3. Co-occurrence network generation

Current curation table displayed in the curation app can be extracted using the `get_curated_table` method.

In [15]:
curated_occurrence_data = curation_app.get_curated_table()
curated_occurrence_data.head(5)

,paper,section,paragraph,aggregated_entities,uid,definition,paper_frequency,entity_type
entity,,,,,,,,
acute respiratory distress syndrome,"{214924, 197804, 129074, 179426, 184360, 211125}","{214924:Introduction, 184360:Ang-(1-7) Analogu...",{179426:Morbidity And Mortality In Diabetic Co...,"[acute respiratory distress syndrome, ards]",http://purl.obolibrary.org/obo/NCIT_C3353,Progressive and life-threatening pulmonary dis...,6,DISEASE
angiotensin-converting enzyme 2,"{214924, 197804, 179426, 184360, 211373, 16056...",{184360:Combined Therapeutic Potential Targeti...,{214924:The Immune Response To Sars-Cov-2 ::: ...,"[ace, ace2, ace2 receptor, ace2 receptors, ang...",http://purl.obolibrary.org/obo/NCIT_C102530,"Angiotensin-converting enzyme 2 (805 aa, ~92 k...",7,PROTEIN
blood,"{214728, 78018, 214924, 197804, 179426, 147333...",{214728:Cap Community-Acquired Pneumonia Covid...,"{211125:Discussion:27, 184360:Gliptins ::: The...",[blood],http://purl.obolibrary.org/obo/NCIT_C12434,A liquid tissue; its major function is to tran...,8,ORGAN
cardiovascular system,"{214924, 197804, 179426, 184360, 160564}","{214924:Introduction, 184360:Gliptins , 214924...","{197804:Discussion:42, 214924:Abstract:1, 1843...",[cardiovascular],http://purl.obolibrary.org/obo/NCIT_C12686,Relating to the heart and the blood vessels or...,5,ORGAN
coronaviridae,"{78018, 214924, 197804, 151932, 184360, 211373...","{151932:Abstract, 214924:Introduction, 211373:...",{184360:Mechanisms Of Sars-Cov-2 Entry Into Ho...,"[coronavirus disease, coronavirus family, coro...",http://purl.obolibrary.org/obo/NCIT_C113205,A family of positive sense ssRNA viruses of zo...,7,ORGANISM


We can also retreive current values of the `Keep` field (these entities will be also included in the resulting co-occurrence network).

In [16]:
curation_app.get_terms_to_include()

['glucose', 'covid-19']

And the number of top frequent entities to use for network generation.

In [17]:
curation_app.n_most_frequent

500

### Generating co-occurrence networks

In the cell below we generate two co-occurrence networks: for paper- and paragraph-based entity co-occurrences. Along with the network generation the `generate_cooccurrence_analysis` function 
- computes node centrality metrics (such as degree, RageRank)
- computes co-occurrence statistics (such as frequency, pointwise mutual information and normalized pointwise mutual information) and assignes them as weights to the corresponding edges
- performs entity community detection based on different co-occurrence statistics
- computes mutual-information-based minimum spanning trees.

In [18]:
%%time
type_data = curated_occurrence_data[["entity_type"]].rename(columns={"entity_type": "type"})

graphs, trees = generate_cooccurrence_analysis(
    curated_occurrence_data,  factor_counts,
    n_most_frequent=curation_app.n_most_frequent if curation_app.n_most_frequent else 100,
    type_data=type_data, 
    factors=["paper", "paragraph"],
    keep=curation_app.get_terms_to_include(),
    cores=10)

print("Done.")

-------------------------------
Factor: paper
-------------------------------
Examining 378 pairs of terms for co-occurrence...
-------------------------------
Factor: paragraph
-------------------------------
Examining 378 pairs of terms for co-occurrence...
Done.
CPU times: user 309 ms, sys: 155 ms, total: 464 ms
Wall time: 865 ms


## 4. Network visualization and analysis

### Loading the generated graphs into the visualization app

In [ ]:
visualization_app.set_graph(
    "Paper-based graph", graphs["paper"],
    tree_object=trees["paper"], default_top_n=100)

visualization_app.set_graph(
    "Paragraph-based graph", graphs["paragraph"],
    tree_object=trees["paragraph"], default_top_n=100)

visualization_app.set_current_graph("Paragraph-based graph")

### Loading papers' meta-data into the app

We now load an additional dataset containing some meta-data on the papers where the entities analyzed in this notebook occur.

In [ ]:
paper_data = pd.read_csv("../data/Glucose_risk_3000_paper_meta_data.csv")
paper_data = paper_data.set_index("id")
paper_data.head(3)

We pass a callback for the lookup of paper meta-data to the visualization app using the `set_list_papers_callback` method.

In [ ]:
def list_papers(paper_data, selected_papers, limit=200):
    selected_paper_data = paper_data.loc[[int(p) for p in selected_papers]].head(200)
    return selected_paper_data.to_dict("records")

visualization_app.set_list_papers_callback(lambda x: list_papers(paper_data, x))

The ontology linking process described above is noisy, therefore, we would like to keep a possibility of accessing, the raw entities that were linked to particular ontology concepts. For this we define the function `get_aggregated_entities` that retreives such raw entities and we pass it to the visualization app using the `set_aggregated_entities_callback` method.

In [ ]:
def top_n(data_dict, n, smallest=False):
    """Return top `n` keys of the input dictionary by their value."""
    df = pd.DataFrame(dict(data_dict).items(), columns=["id", "value"])
    if smallest:
        df = df.nsmallest(n, columns=["value"])
    else:
        df = df.nlargest(n, columns=["value"])
    return(list(df["id"]))


def get_aggregated_entities(entity, n):
    if "aggregated_entities" in curated_occurrence_data.columns:
        aggregated = curated_occurrence_data.loc[entity]["aggregated_entities"]
    else:
        aggregated = [entity]
    if curation_input_table is not None:
        df = curation_input_table.set_index("entity")
        if entity in curated_occurrence_data.index:
            freqs = df.loc[aggregated]["paper_frequency"].to_dict()
        else:
            return {}
    else:
        df = data.copy()
        df["entity"] = data["entity"].apply(lambda x: x.lower())
        freqs = df[df["entity"].apply(lambda x: x.lower() in aggregated)].groupby("entity").aggregate(
            lambda x: len(x))["entity_type"].to_dict()
    if len(freqs) == 0:
        return {}
    return {e: freqs[e] for e in top_n(freqs, n)}

visualization_app.set_aggregated_entities_callback(
    lambda x: get_aggregated_entities(x, 10))

Finally, we create a dictionary `definitions` that will serve the visualization app as the lookup table for accessing the definitions of different ontology concepts.

In [ ]:
definitions = ontology_linking[["concept", "definition"]].groupby(
    "concept").aggregate(lambda x: list(x)[0]).to_dict()["definition"]
visualization_app.set_entity_definitons(definitions)

### Launching the visualization app

As before, the interactive graph visualization app can be launched in two modes: inline and external. Here we recommend the external mode for better user experience.

In [ ]:
visualization_app.run(port=8082, mode="external")